In [ ]:
!pip install google-play-scraper

  Obtaining dependency information for google-play-scraper from https://files.pythonhosted.org/packages/33/f7/a23ef3cf8efc9ab3aee565971f59906811e6ce95475314ef7b18d02f30ba/google_play_scraper-1.2.7-py3-none-any.whl.metadata
  Using cached google_play_scraper-1.2.7-py3-none-any.whl.metadata (50 kB)
Using cached google_play_scraper-1.2.7-py3-none-any.whl (28 kB)



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from google_play_scraper import Sort, reviews
from datetime import datetime
import time

def scrape_goride_reviews(year=2024, month=1, keywords=None):
    if keywords is None:
        keywords = ["goride", "Goride", "GoRide", "go-ride", "GO RIDE", "ride"]
    
    app_id = 'com.gojek.app'
    start_date = datetime(year, month, 1)
    end_date = datetime(year + int(month == 12), (month % 12) + 1, 1)
    
    all_reviews, token = [], None
    while True:
        result, token = reviews(
            app_id,
            lang='id',
            country='id',
            sort=Sort.NEWEST,
            count=200,
            continuation_token=token
        )
        
        for r in result:
            if r['at'] < start_date:
                token = None
                break
            if start_date <= r['at'] < end_date and any(k in r['content'] for k in keywords):
                all_reviews.append({
                    'review_id': r['reviewId'],
                    'username': r['userName'],
                    'rating': r['score'],
                    'review_text': r['content'],
                    'date': r['at'].strftime('%Y-%m-%d'),
                    'likes': r['thumbsUpCount']
                })
        if not token:
            break
    
    df = pd.DataFrame(all_reviews)
    filename = f"ulasan_goride_{year}_{month:02d}.csv"
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"{len(df)} ulasan disimpan di {filename}")
    return df

# Contoh pemanggilan
if __name__ == "__main__":
    df = scrape_goride_reviews(year=2024, month=4)
    print(df.head())


TypeError: argument of type 'NoneType' is not iterable

In [6]:
import pandas as pd
from google_play_scraper import Sort, reviews
from datetime import datetime
import time

def scrape_goride_reviews():
    """
    Fungsi untuk mengambil ulasan GoRide dari Google Play Store pada bulan tertentu di tahun tertentu.
    Menggunakan pendekatan batch processing dengan continuation token.
    """
    
    # =============================================
    # KONFIGURASI YANG DAPAT DIUBAH
    # =============================================
    TARGET_YEAR = 2025    # Tahun target
    TARGET_MONTH = 4     # Bulan target (1-12)
    BATCH_SIZE = 200      # Jumlah ulasan per batch
    LANG = 'id'           # Bahasa ulasan
    COUNTRY = 'id'        # Negara ulasan
    APP_ID = 'com.gojek.app'  # ID aplikasi Gojek
    
    # Kata kunci untuk filter ulasan GoRide
    KEYWORDS = ["Goride", "goride", "go-ride", "Go-ride", "GO RIDE", "go ride", "ride", "Ride"]
    # =============================================
    
    # Validasi input bulan
    if TARGET_MONTH < 1 or TARGET_MONTH > 12:
        print("Error: Bulan harus antara 1 (Januari) hingga 12 (Desember)")
        return None
    
    # Konversi bulan ke nama bulan untuk display
    month_names = [
        "Januari", "Februari", "Maret", "April", "Mei", "Juni",
        "Juli", "Agustus", "September", "Oktober", "November", "Desember"
    ]
    month_name = month_names[TARGET_MONTH - 1]
    
    print(f"\nMemulai scraping ulasan GoRide untuk bulan {month_name} {TARGET_YEAR}...")
    print(f"Menggunakan kata kunci: {', '.join(KEYWORDS)}")
    
    # Hitung range tanggal target
    start_date = datetime(TARGET_YEAR, TARGET_MONTH, 1)
    if TARGET_MONTH == 12:
        end_date = datetime(TARGET_YEAR + 1, 1, 1)
    else:
        end_date = datetime(TARGET_YEAR, TARGET_MONTH + 1, 1)
    
    print(f"Periode target: {start_date.strftime('%Y-%m-%d')} hingga {end_date.strftime('%Y-%m-%d')}")
    
    # Kumpulkan ulasan
    all_reviews = []
    continuation_token = None
    batch_count = 0
    total_found = 0
    stop_early = False
    
    start_time = time.time()
    
    while True:
        batch_count += 1
        try:
            print(f"\nMengambil batch ke-{batch_count}...")
            
            # Ambil batch ulasan
            result, continuation_token = reviews(
                APP_ID,
                lang=LANG,
                country=COUNTRY,
                sort=Sort.NEWEST,
                count=BATCH_SIZE,
                continuation_token=continuation_token
            )
            
            batch_found = 0
            for review in result:
                review_date = review['at']
                review_text = review['content']
                
                # Cek apakah sudah melewati periode target
                if review_date < start_date:
                    print("Sudah mencapai periode sebelum target, menghentikan proses...")
                    stop_early = True
                    break
                
                # Cek apakah dalam periode target dan mengandung kata kunci
                if (review_text and start_date <= review_date < end_date and 
                    any(keyword in review_text for keyword in KEYWORDS)):
                    
                    all_reviews.append({
                        'review_id': review['reviewId'],
                        'user_name': review['userName'],
                        'rating': review['score'],
                        'review_text': review_text,
                        'date': review_date.strftime('%Y-%m-%d %H:%M:%S'),
                        'likes': review['thumbsUpCount'],
                        'app_version': review.get('reviewCreatedVersion', '')
                    })
                    batch_found += 1
            
            total_found += batch_found
            print(f"Ditemukan {batch_found} ulasan yang sesuai dalam batch ini")
            print(f"Total sementara: {total_found} ulasan")
            
            # Berhenti jika tidak ada token lanjutan atau sudah melewati periode
            if not continuation_token or stop_early:
                break
                
        except Exception as e:
            print(f"Error dalam pengambilan data: {str(e)}")
            break
    
    elapsed_time = time.time() - start_time
    
    if not all_reviews:
        print("\nTidak ada ulasan GoRide yang ditemukan untuk periode ini.")
        return None
    
    # Konversi ke DataFrame
    df = pd.DataFrame(all_reviews)
    
    # Simpan ke file CSV
    csv_filename = f"ulasan_goride_{TARGET_YEAR}_{TARGET_MONTH:02d}.csv"
    df.to_csv(csv_filename, index=False, encoding='utf-8')
    
    print("\nRingkasan Hasil:")
    print(f"Total waktu eksekusi: {elapsed_time:.2f} detik")
    print(f"Total batch diproses: {batch_count}")
    print(f"Total ulasan yang sesuai: {len(df)}")
    print(f"Data berhasil disimpan dalam file: {csv_filename}")
    
    return df

if __name__ == "__main__":
    print("Scraper Ulasan GoRide Google Play Store")
    print("--------------------------------------")
    print("NOTE: Untuk mengubah bulan/tahun, edit variabel TARGET_YEAR dan TARGET_MONTH dalam fungsi scrape_goride_reviews()")
    
    reviews_data = scrape_goride_reviews()
    
    if reviews_data is not None:
        print("\nPreview data:")
        print(reviews_data.head())

Scraper Ulasan GoRide Google Play Store
--------------------------------------
NOTE: Untuk mengubah bulan/tahun, edit variabel TARGET_YEAR dan TARGET_MONTH dalam fungsi scrape_goride_reviews()

Memulai scraping ulasan GoRide untuk bulan April 2025...
Menggunakan kata kunci: Goride, goride, go-ride, Go-ride, GO RIDE, go ride, ride, Ride
Periode target: 2025-04-01 hingga 2025-05-01

Mengambil batch ke-1...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-2...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-3...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-4...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-5...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-6...
Ditemukan 0 ulasan yang sesuai dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-7...
Dite

In [1]:
import pandas as pd
from google_play_scraper import Sort, reviews
from datetime import datetime
import time

def scrape_all_reviews():
    """
    Fungsi untuk mengambil semua ulasan dari Google Play Store pada bulan November 2024.
    Menggunakan pendekatan batch processing dengan continuation token.
    """
    
    # =============================================
    # KONFIGURASI YANG DAPAT DIUBAH
    # =============================================
    TARGET_YEAR = 2024    # Tahun target
    TARGET_MONTH = 12     # Bulan target (November)
    BATCH_SIZE = 200      # Jumlah ulasan per batch
    LANG = 'id'           # Bahasa ulasan
    COUNTRY = 'id'        # Negara ulasan
    APP_ID = 'com.gojek.app'  # ID aplikasi Gojek
    # =============================================
    
    # Validasi input bulan
    if TARGET_MONTH < 1 or TARGET_MONTH > 12:
        print("Error: Bulan harus antara 1 (Januari) hingga 12 (Desember)")
        return None
    
    # Konversi bulan ke nama bulan untuk display
    month_names = [
        "Januari", "Februari", "Maret", "April", "Mei", "Juni",
        "Juli", "Agustus", "September", "Oktober", "November", "Desember"
    ]
    month_name = month_names[TARGET_MONTH - 1]
    
    print(f"\nMemulai scraping semua ulasan untuk bulan {month_name} {TARGET_YEAR}...")
    
    # Hitung range tanggal target
    start_date = datetime(TARGET_YEAR, TARGET_MONTH, 1)
    if TARGET_MONTH == 12:
        end_date = datetime(TARGET_YEAR + 1, 1, 1)
    else:
        end_date = datetime(TARGET_YEAR, TARGET_MONTH + 1, 1)
    
    print(f"Periode target: {start_date.strftime('%Y-%m-%d')} hingga {end_date.strftime('%Y-%m-%d')}")
    
    # Kumpulkan ulasan
    all_reviews = []
    continuation_token = None
    batch_count = 0
    total_found = 0
    stop_early = False
    
    start_time = time.time()
    
    while True:
        batch_count += 1
        try:
            print(f"\nMengambil batch ke-{batch_count}...")
            
            # Ambil batch ulasan
            result, continuation_token = reviews(
                APP_ID,
                lang=LANG,
                country=COUNTRY,
                sort=Sort.NEWEST,
                count=BATCH_SIZE,
                continuation_token=continuation_token
            )
            
            batch_found = 0
            for review in result:
                review_date = review['at']
                
                # Cek apakah sudah melewati periode target
                if review_date < start_date:
                    print("Sudah mencapai periode sebelum target, menghentikan proses...")
                    stop_early = True
                    break
                
                # Cek apakah dalam periode target
                if start_date <= review_date < end_date:
                    all_reviews.append({
                        'review_id': review['reviewId'],
                        'user_name': review['userName'],
                        'rating': review['score'],
                        'review_text': review['content'],
                        'date': review_date.strftime('%Y-%m-%d %H:%M:%S'),
                        'likes': review['thumbsUpCount'],
                        'app_version': review.get('reviewCreatedVersion', '')
                    })
                    batch_found += 1
            
            total_found += batch_found
            print(f"Ditemukan {batch_found} ulasan dalam batch ini")
            print(f"Total sementara: {total_found} ulasan")
            
            # Berhenti jika tidak ada token lanjutan atau sudah melewati periode
            if not continuation_token or stop_early:
                break
                
        except Exception as e:
            print(f"Error dalam pengambilan data: {str(e)}")
            break
    
    elapsed_time = time.time() - start_time
    
    if not all_reviews:
        print("\nTidak ada ulasan yang ditemukan untuk periode ini.")
        return None
    
    # Konversi ke DataFrame
    df = pd.DataFrame(all_reviews)
    
    # Simpan ke file CSV
    csv_filename = f"semua_ulasannnnnn_{TARGET_YEAR}_{TARGET_MONTH:02d}.csv"
    df.to_csv(csv_filename, index=False, encoding='utf-8')
    
    print("\nRingkasan Hasil:")
    print(f"Total waktu eksekusi: {elapsed_time:.2f} detik")
    print(f"Total batch diproses: {batch_count}")
    print(f"Total ulasan yang ditemukan: {len(df)}")
    print(f"Data berhasil disimpan dalam file: {csv_filename}")
    
    return df

if __name__ == "__main__":
    print("Scraper Ulasan Google Play Store")
    print("--------------------------------------")
    print("NOTE: Script ini akan mengambil SEMUA ulasan pada November 2024")
    
    reviews_data = scrape_all_reviews()
    
    if reviews_data is not None:
        print("\nPreview data:")
        print(reviews_data.head())

Scraper Ulasan Google Play Store
--------------------------------------
NOTE: Script ini akan mengambil SEMUA ulasan pada November 2024

Memulai scraping semua ulasan untuk bulan November 2024...
Periode target: 2024-11-01 hingga 2024-12-01

Mengambil batch ke-1...
Ditemukan 0 ulasan dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-2...
Ditemukan 0 ulasan dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-3...
Ditemukan 0 ulasan dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-4...
Ditemukan 0 ulasan dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-5...
Ditemukan 0 ulasan dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-6...
Ditemukan 0 ulasan dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-7...
Ditemukan 0 ulasan dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-8...
Ditemukan 0 ulasan dalam batch ini
Total sementara: 0 ulasan

Mengambil batch ke-9...
Ditemukan 0 ulasan dalam batch ini
Total semen

In [1]:
# ✅ STEP 2: Import library
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

C:\Users\adrea\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
# ✅ STEP 3: Load IndoBERT model dan tokenizer
model_name = "mdhugol/indonesia-bert-sentiment-classification"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [3]:
# ✅ STEP 4: Load dataset CSV kamu


df = pd.read_csv("goride_review.csv")
print(f"Jumlah ulasan: {len(df)}")
df.head()

Jumlah ulasan: 659


,review_id,user_name,rating,review_text,date,likes,app_version
0,559a1ccb-6e0a-44ff-87d2-a1432955390b,Ayun Hawa,1,"Aneh, voucher goride gocar gamuncul semua pada...",2024-01-31 16:39:09,0,NaN
1,54b1e0cf-a2f9-4732-82c2-b18244afe52e,supri yanto,5,Kok buka aplikasi goride hilang2 mulu...,2024-01-31 13:37:36,0,4.81.2
2,506c4016-0dbb-4b58-b754-9e5b1c22a0ee,Ruwy Masyari,5,Gojek keren bisa pesen sekalian 2 goride satu ...,2024-01-30 14:29:51,0,NaN
3,cdbf51ee-e2d8-4137-8c71-06177c9c5d49,Achmad Sayid,3,Sekarang udah mahal goride dan gofood,2024-01-30 09:18:49,0,4.81.1
4,93035bc2-aaf6-4462-927f-76c57a7f4237,Resa Agustin,1,Kesalahan dari sistem gojek saldo Gopay kepoto...,2024-01-29 23:35:04,0,4.67.2


In [4]:
# ✅ STEP 5: Buat pipeline analisis sentimen
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

Device set to use cpu


In [5]:
# ✅ STEP 6: Lakukan pseudo-labeling (berlabel otomatis)
# Proses batch untuk menghindari error terlalu banyak permintaan
batch_size = 100
labels = []

for i in range(0, len(df), batch_size):
    batch = df["review_text"].iloc[i:i+batch_size].tolist()
    preds = sentiment_pipeline(batch)
    labels.extend([pred["label"] for pred in preds])

df["pseudo_label"] = labels

In [6]:
# ✅ STEP 7: Simpan hasil ke file baru
output_filename = "ulasan_berlabel1.csv"
df.to_csv(output_filename, index=False)
print(f"✅ Berhasil disimpan: {output_filename}")

# # ✅ STEP 8: Download hasil ke lokal
# files.download(output_filename)

✅ Berhasil disimpan: ulasan_berlabel1.csv


In [8]:
print(model.config.id2label)


{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}


In [9]:
import pandas as pd

# Ganti ini dengan nama file hasil pseudo-labeling kamu
input_file = "ulasan_berlabel1.csv"
output_file = "pseudo_labeled_angka.csv"

# Baca CSV
df = pd.read_csv(input_file)

# Pastikan kolom label ada
if 'pseudo_label' not in df.columns:
    raise ValueError("Kolom 'label' tidak ditemukan dalam file. Pastikan nama kolomnya sesuai.")

# Mapping LABEL ke angka
label_map = {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
}

# Ubah label
df['pseudo_label'] = df['pseudo_label'].map(label_map)

# Simpan ke file baru
df.to_csv(output_file, index=False)
print(f"✅ Label berhasil dikonversi dan disimpan ke: {output_file}")


✅ Label berhasil dikonversi dan disimpan ke: pseudo_labeled_angka.csv


In [10]:
import pandas as pd

# Load dataset
df = pd.read_csv('pseudo_labeled_angka.csv')

# Tampilkan 5 baris pertama untuk melihat strukturnya
df.head()


,review_id,user_name,rating,review_text,date,likes,app_version,pseudo_label
0,559a1ccb-6e0a-44ff-87d2-a1432955390b,Ayun Hawa,1,"Aneh, voucher goride gocar gamuncul semua pada...",2024-01-31 16:39:09,0,NaN,2
1,54b1e0cf-a2f9-4732-82c2-b18244afe52e,supri yanto,5,Kok buka aplikasi goride hilang2 mulu...,2024-01-31 13:37:36,0,4.81.2,2
2,506c4016-0dbb-4b58-b754-9e5b1c22a0ee,Ruwy Masyari,5,Gojek keren bisa pesen sekalian 2 goride satu ...,2024-01-30 14:29:51,0,NaN,0
3,cdbf51ee-e2d8-4137-8c71-06177c9c5d49,Achmad Sayid,3,Sekarang udah mahal goride dan gofood,2024-01-30 09:18:49,0,4.81.1,2
4,93035bc2-aaf6-4462-927f-76c57a7f4237,Resa Agustin,1,Kesalahan dari sistem gojek saldo Gopay kepoto...,2024-01-29 23:35:04,0,4.67.2,2


In [17]:
# Hapus baris dengan label 1 (neutral) dan 2 (negatif)
df_filtered = df[df['pseudo_label'] == 1]

# Cek jumlah data yang tersisa
print(f"Jumlah data setelah difilter: {len(df_filtered)}")




Jumlah data setelah difilter: 87


In [18]:
# # Simpan ke file baru (opsional)
df_filtered.to_csv('only_neutral_reviews.csv', index=False)

In [20]:
# Load file kedua untuk diacak
df2 = pd.read_csv("Data2.csv")

# Acak seluruh baris data dan reset index
df2_shuffled = df2.sample(frac=1, random_state=42).reset_index(drop=True)

# Simpan hasil yang sudah diacak
shuffled_path2 = "fix.csv"
df2_shuffled.to_csv(shuffled_path2, index=False)

# Tampilkan beberapa baris awal dan akhir
df2_shuffled.head(), df2_shuffled.tail()

(                 user_name  rating  \
 0           Eko Hermawanto       1   
 1           annennta nenta       3   
 2  2Firmansyah 6Firmansyah       5   
 3         Emmy Simanjuntak       5   
 4                  Man Cha       5   
 
                                          review_text                 date  \
 0  Matikan Fitur Order Dobel/2 pemesanan GoRide. ...  2024-08-31 10:25:33   
 1  Tolong ya..Pleeeeease...buat CUSTOMER yg Pesen...  2024-03-14 17:06:06   
 2  Saya harap di update kedepanya saat pesan go f...  2024-11-20 10:14:58   
 3  Buatku yg tinggal agak jauh dr kota,ini aplika...  2024-10-04 21:36:13   
 4  Layanan Goride sangat membantu untuk aktifitas...  2024-12-18 23:55:10   
 
    label  
 0      0  
 1      0  
 2      1  
 3      1  
 4      1  ,
                   user_name  rating  \
 1031  Muhammad Nazril ilham       5   
 1032              NENA SUTA       5   
 1033       Krisnoto Suherno       1   
 1034            Aprilla Ayu       1   
 1035          Nurul 

In [21]:
import pandas as pd

# Membaca file CSV
df = pd.read_csv('fix.csv')

# Menambahkan kolom 'sentiment' berdasarkan nilai kolom 'label'
df['sentiment'] = df['label'].map({0: 'Negatif', 1: 'Positif'})

# Menampilkan beberapa baris pertama untuk memastikan
print(df.head())

# Menyimpan hasil ke file baru (opsional)
df.to_csv('fix_with_sentiment.csv', index=False)


                 user_name  rating  \
0           Eko Hermawanto       1   
1           annennta nenta       3   
2  2Firmansyah 6Firmansyah       5   
3         Emmy Simanjuntak       5   
4                  Man Cha       5   

                                         review_text                 date  \
0  Matikan Fitur Order ganda pemesanan GoRide. Se...  2024-08-31 10:25:33   
1  Tolong ya..Pleeeeease...buat CUSTOMER yg Pesen...  2024-03-14 17:06:06   
2  Saya harap di update kedepanya saat pesan go f...  2024-11-20 10:14:58   
3  Buatku yg tinggal agak jauh dr kota,ini aplika...  2024-10-04 21:36:13   
4  Layanan Goride sangat membantu untuk aktifitas...  2024-12-18 23:55:10   

   label sentiment  
0      0   Negatif  
1      0   Negatif  
2      1   Positif  
3      1   Positif  
4      1   Positif  
